<a href="https://colab.research.google.com/github/denikoer/aqi/blob/master/IoTMasterClass_ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Import**

In [ ]:
import gspread
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from google.colab import auth
from google.auth import default


### **Open/Load Sample Data**

In [ ]:
# auth/load google credentials
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet by its URL (ensure it's publicly accessible)
sheet_url = 'https://docs.google.com/spreadsheets/d/1SLwix8SFx3VUShDSNgh8-uu1h2f-UulPTu2t5CXWRnM/edit'  # Replace with your Google Sheet URL

# Authenticate and open the spreadsheet
spreadsheet = gc.open_by_url(sheet_url)

# Select a specific worksheet
worksheet = spreadsheet.worksheet("sample-data-fan-control")

# Get all values from the worksheet
values = worksheet.get_all_values()

# Convert to a Pandas DataFrame
df = pd.DataFrame(values[1:], columns=values[0])  # Assuming the first row contains column headers

### **Process Data**

In [ ]:
# clean up data
weekday_map = {'sunday': 0, 'monday': 1, 'tuesday': 2, 'wednesday': 3, 'thursday': 4, 'friday': 5, 'saturday': 6}
df['day'] = df['day'].map(weekday_map)

truth_map = {'FALSE': 0, 'TRUE': 1}
df['studio_motion'] = df['studio_motion'].map(truth_map)
df['dog_motion'] = df['dog_motion'].map(truth_map)

on_off_map = {'off': 0, 'on': 1}
df['studio_fan'] = df['studio_fan'].map(on_off_map)
df['dog_fan'] = df['dog_fan'].map(on_off_map)

# Function to convert time stamp string to seconds
def time_to_seconds(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    return hours * 3600 + minutes * 60 + seconds

# Convert the 'Timestamps' column to seconds in day
df['time'] = df['time'].apply(time_to_seconds)

NameError: ignored

In [ ]:
df

In [ ]:
# seperate studio fan and dog fan  data
# select studio fan and dog fan state DataFrame
df_st = df[['time','day','studio_motion', 'studio_temp','studio_fan']]
df_do = df[['time','day','dog_motion', 'dog_temp','dog_fan']]

NameError: ignored

In [ ]:
df_st

In [ ]:
df_do

In [ ]:
# seperate features/Inputs (X) and targets (classes: y)

# condition analysis (target data, Class)
y_st = df_st['studio_fan']
y_do = df_do['dog_fan']

# independent variables (Features/Input)
X_st = df_st.drop(['studio_fan'], axis = 1)
X_do = df_do.drop(['dog_fan'], axis = 1)

NameError: ignored

In [ ]:
# split training and test data

X_train_st, X_test_st, y_train_st, y_test_st = train_test_split(X_st, y_st, test_size=0.20, random_state=91)
X_train_do, X_test_do, y_train_do, y_test_do = train_test_split(X_do, y_do, test_size=0.20, random_state=91)

### **Train Data**

In [ ]:
# init and train RandomForestClassifier for Studio Fan Model

# time, day,	studio_motion,	studio_temp
class_weights = {0: 10, 1: 27, 2: 4, 3: 40}  # Data Tuning: Assign a higher weight to classes with higher importance

model_st = RandomForestClassifier(oob_score=True, max_depth=8, random_state=90, class_weight=class_weights)
model_st.fit(X_train_st,y_train_st)

# Calculate estimated OOB (Out Of Bag) score
""" The OOB score can serve as a useful estimate of how well your random forest model is likely
to perform on unseen data without the need for a separate validation set.
However, it's still a good practice to use additional evaluation techniques
like cross-validation to assess the model's performance thoroughly. """

print(f'Studio Fan OOB SCORE: {model_st.oob_score_}')

Studio Fan OOB SCORE: 0.970873786407767


In [ ]:
# init and train RandomForestClassifier for Dog Fan Model

# time, day,	studio_motion,	studio_temp
class_weights = {0: 30, 1: 20, 2: 4, 3: 40}  # Data Tuning: Example: Assign a higher weight (5) to class 1

model_do = RandomForestClassifier(oob_score=True, max_depth=8, random_state=91, class_weight=class_weights)
model_do.fit(X_train_do,y_train_do)

# Calculate estimated OOB (Out Of Bag) score
""" The OOB score can serve as a useful estimate of how well your random forest model is likely
to perform on unseen data without the need for a separate validation set.
However, it's still a good practice to use additional evaluation techniques
like cross-validation to assess the model's performance thoroughly. """

print(f'Dog Fan OOB SCORE: {model_do.oob_score_}')

Dog Fan OOB SCORE: 0.9597780859916782


### **Validate Tune Data**

In [ ]:
# Calculate Accuracy
from sklearn.metrics import accuracy_score, classification_report

# calculate studio fan state accuracy
y_pred_st = model_st.predict(X_test_st)
accuracy_st = accuracy_score(y_test_st, y_pred_st)
print(f'Studio Fan Accuracy: {accuracy_st}')

# calculate dog fan state accuracy
y_pred_do = model_do.predict(X_test_do)
accuracy_do = accuracy_score(y_test_do, y_pred_do)
print(f'Dog Fan Accuracy: {accuracy_do}')

NameError: ignored

In [ ]:
# calculate and visualize the confusion matrix
""" A confusion matrix is a table used in machine learning to evaluate the
performance of a classification model, showing true positives, true negatives,
false positives, and false negatives."""

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_test, y_pred, label_name):
  # Generate a confusion matrix
  confusion = confusion_matrix(y_test, y_pred)

  # Calculate percentages for each cell in the confusion matrix
  total_samples = np.sum(confusion)
  confusion_percent = (confusion / total_samples) * 100

  # Plot the confusion matrix with percentages
  plt.figure(figsize=(4, 3))
  sns.heatmap(confusion_percent, annot=True, fmt='.2f', cmap='Blues', cbar=False, square=True)
  plt.xlabel('Predicted Labels : ' + label_name)
  plt.ylabel('Actual Labels')
  plt.title('Confusion Matrix (Percentages)')
  plt.show()


plot_confusion_matrix(y_test_st, y_pred_st, "Studio Fan Model")
plot_confusion_matrix(y_test_do, y_pred_do, "Dog Fan Model")



### **Download and Deploy Model**

In [ ]:
import joblib
from google.colab import files

# Download Studio Fan Model
joblib.dump(model_st, "studio_fan_model.joblib")
files.download("studio_fan_model.joblib")

NameError: ignored

In [ ]:
import joblib
from google.colab import files

# Download Studio Fan Model
joblib.dump(model_st, "dog_fan_model.joblib")
files.download("dog_fan_model.joblib")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>